In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageChops, ImageEnhance

In [4]:
import cv2
import os
import numpy as np
from PIL import Image, ImageEnhance
import tempfile

def error_level_analysis(image, quality=90):
    """Perform ELA using bitwise XOR instead of absolute difference."""
    try:
        im = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Save to a temporary JPEG file
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp:
            temp_compressed = tmp.name
            im.save(temp_compressed, "JPEG", quality=quality)

        # Reload and convert compressed image
        compressed = Image.open(temp_compressed)
        os.remove(temp_compressed)

        # Convert both to NumPy arrays
        im_np = np.array(im)
        compressed_np = np.array(compressed)

        # Apply XOR difference (bitwise)
        xor_diff = cv2.bitwise_xor(im_np, compressed_np)

        # Enhance brightness for visualization
        xor_image = Image.fromarray(xor_diff)
        xor_image = ImageEnhance.Brightness(xor_image).enhance(5)

        return cv2.cvtColor(np.array(xor_image), cv2.COLOR_RGB2BGR)

    except Exception as e:
        print(f"ELA-XOR Error: {e}")
        return None
    
def preprocess_image(image, image_size=(128, 128)):
    ela_image = error_level_analysis(image)
    if ela_image is None:
        return None
    ela_resized = cv2.resize(ela_image, image_size)
    gray = cv2.cvtColor(ela_resized, cv2.COLOR_BGR2GRAY)
    return gray.reshape(image_size[0], image_size[1], 1)


In [5]:
def prepare_dataset(dataset_path, image_size=(128, 128), data_limit=None):
    """Load and preprocess images, returning feature matrix X and labels y."""
    X, y = [], []
    for class_name in ["Au", "Tp"]:
        class_path = os.path.join(dataset_path, class_name)
        label = 0 if class_name == "Au" else 1
        data_list = (
            os.listdir(class_path)[0:data_limit]
            if data_limit
            else os.listdir(class_path)
        )
        for img_file in data_list:
            if img_file.lower().endswith(
                (".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp")
            ):
                img = cv2.imread(os.path.join(class_path, img_file))
                processed_img = preprocess_image(img, image_size)
                if processed_img is not None:
                    X.append(processed_img)
                    y.append(label)
    return np.array(X), np.array(y)

In [6]:


# Define the CNN model
def build_model(input_shape=(128, 128, 1), num_classes=2):
    model = models.Sequential()

    # First convolutional layer: 32 filters, 5x5 kernel
    model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))

    # Second convolutional layer: 32 filters, 5x5 kernel
    model.add(layers.Conv2D(32, (5, 5), activation='relu'))

    # Max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Dropout to prevent overfitting
    model.add(layers.Dropout(0.25))

    # Flatten layer to convert 2D feature maps to 1D feature vector
    model.add(layers.Flatten())

    # Fully connected (dense) layer
    model.add(layers.Dense(128, activation='relu'))

    # Output layer with softmax activation
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model with RMSprop optimizer
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model



In [7]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def train_model(model, X, y, epochs=12):
    """Train the model with optimized parameters and class weighting."""

    # One-hot encode the labels
    y = to_categorical(y, num_classes=2)

    # Train/val/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y.argmax(axis=1), random_state=42
    )
    print(X_train.shape, y_train.shape)

    X_test, X_val, y_test, y_val = train_test_split(
        X_test, y_test, test_size=0.5, stratify=y_test.argmax(axis=1), random_state=42
    )
    print(X_test.shape, y_test.shape)
    print(X_val.shape, y_val.shape)

    # Normalize pixel values
    X_train = X_train.astype("float32") / 255.0
    X_val = X_val.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0

    # Train the model
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
    )
    return model, history, X_test, y_test


In [ ]:
def evaluate_model(model, X_test, y_test):
    """Evaluate the model on the test set and generate various plots."""
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    print("\nTest Metrics:")
    print(classification_report(y_test, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=["Non-Forged", "Forged"],
        yticklabels=["Non-Forged", "Forged"],
    )
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure()
    # plt.plot(fpr, tpr, label=f"ROC curve (AUC = {test_auc:.2f})")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.figure()
    plt.plot(recall, precision, label="Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.show()